# **IK for a Mobile Manipulator**

In this notebook, we will solve inverse kinematics for one arm on a PR2 robot. This robot model has been modified to have convex collision geometries, and to simplify this problem, we have welded (fixed) several joints that are irrelevant to the kinematics of the arms.

**Learning Objectives**
- Get more practice implementing constraints in the `InverseKinematics` class in Drake
- Investigate how optimization for IK changes with a mobile base

**What You'll Implement**
- IK for a mobile manipulator using the InverseKinematics class in Drake
- Constraints that place the left gripper of the PR2 at a desired pose

## Setup and Imports

Let us first import our standard drake functionality

In [5]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Diagram,
    DiagramBuilder,
    DiscreteContactApproximation,
    InverseKinematics,
    Joint,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Solve,
    StartMeshcat,
    WeldJoint,
    eq,
)

from manipulation import ConfigureParser, running_as_notebook

In [6]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [7]:
def ReplaceJointWithWeld(plant: MultibodyPlant, joint: Joint) -> None:
    for actuator in [
        plant.get_joint_actuator(index) for index in plant.GetJointActuatorIndices()
    ]:
        if actuator.joint() == joint:
            plant.RemoveJointActuator(actuator)

    weld = WeldJoint(
        joint.name(), joint.frame_on_parent(), joint.frame_on_child(), RigidTransform()
    )
    plant.RemoveJoint(joint)
    plant.AddJoint(weld)


def build_env() -> tuple[Diagram, MultibodyPlant, SceneGraph]:
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/pr2_shelves.dmd.yaml")
    # Remove mimic constraints:
    for id in plant.GetConstraintIds():
        plant.RemoveConstraint(id)
    # Remove some superfluous joints
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_pan_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_tilt_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_tip_joint"))
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

## Part 1: Implementing IK for the PR2

**YOUR TASK:** Given a `RigidTransform` X_WG, compute a robot configuration placing the left gripper at that pose. We use optimization to solve the IK problem, and we repeatedly solve the program with random initializations until it succeeds. We have implemented a skeleton of the necessary code in the following function, but you must complete several pieces:

- Add position and orientation constraints to the gripper frame.
    - The end effector should match the desired pose to within 1mm translation along each axis
    - The rotation should be off by no more than 1 degree.
- If `fix_base` is True, constrain the base pose $(x,y,\theta)$ to be equal to `base_pose`.
- Add a collision free constraint with [AddMinimumDistanceLowerBoundConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a2ecd71efd675a7e1a4293adb05c9b9df). The minimum distance between any pair of collision geometries should be at least 1cm.
- Compute a random initial guess for the joint angles within the robot's joint limits. You can access the joint limits from the multibody plant, but some of the joints are angle-valued and don't have limits. For these joints, use the range $[-\pi,\pi]$.

In this exercise, we will be using the `InverseKinematics` class in Drake. Take a look at the documentation and types of constraints we can use here: [IK Drake Documentation](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html). It may be helpful to go through the the Door Opening Notebook (exercise 6.1 in the manipulation textbook) before completing the rest of this notebook.

In [8]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-0.83, 0.18, 1.4])
goal_pose = RigidTransform(goal_rotation, goal_position)

In [ ]:
def solve_ik(
    X_WG: RigidTransform,
    max_tries: int = 10,
    fix_base: bool = False,
    base_pose: np.ndarray | None = None,
) -> np.ndarray | None:
    if base_pose is None:
        base_pose = np.zeros(3)

    np.random.seed(16)  # we use this for verification - do not modify it

    diagram, plant, scene_graph = build_env()

    gripper_frame = plant.GetFrameByName("l_gripper_palm_link")
    world_frame = plant.world_frame()

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(np.eye(len(q_variables)), q_nominal, q_variables)

    # TODO: Add your constraints here
    # HINT: Use AddPositionConstraint and AddOrientationConstraint like in the door opening exercise

    # Position constraint
    position_tol = 0.001
    lower_bound = X_WG.translation() - position_tol * np.ones(3)
    upper_bound = X_WG.translation() + position_tol * np.ones(3)
    ik.AddPositionConstraint(
        gripper_frame,
        np.array([0, 0, 0]),
        world_frame,
        lower_bound,
        upper_bound,
    )

    # Orientation constraint
    orientation_tol = np.pi / 180
    ik.AddOrientationConstraint(
        world_frame,
        X_WG.rotation(),
        gripper_frame,
        RotationMatrix.Identity(),
        orientation_tol,
    )

    # Add collision avoidance constraint
    ik.AddMinimumDistanceLowerBoundConstraint(0.01)  # 1cm minimum distance

    # Add base pose constraint if fix_base is True
    if fix_base:
        for i in range(3):
            prog.AddLinearEqualityConstraint(q_variables[i] == base_pose[i])

    for count in range(max_tries):
        # Compute a random initial guess here, within the joint limits of the robot
        q_guess = np.zeros(len(q_variables))
        
        joint_indices = plant.GetJointIndices()
        for joint_index in joint_indices:
            joint = plant.get_joint(joint_index)
            
            # Skip joints with no degrees of freedom
            if joint.num_positions() == 0:
                continue
                
            # Get the position start index for this joint in the q vector
            pos_start = joint.position_start()
            num_pos = joint.num_positions()
            
            # Set random values for each position of this joint
            for j in range(num_pos):
                q_idx = pos_start + j
                if q_idx < len(q_variables):
                    try:
                        lower_limits = joint.position_lower_limits()
                        upper_limits = joint.position_upper_limits()
                        
                        if np.isfinite(lower_limits[j]) and np.isfinite(upper_limits[j]):
                            q_guess[q_idx] = np.random.uniform(lower_limits[j], upper_limits[j])
                        else:
                            q_guess[q_idx] = np.random.uniform(-np.pi, np.pi)
                    except:
                        q_guess[q_idx] = np.random.uniform(-np.pi, np.pi)
        
        # Set the initial guess
        prog.SetInitialGuess(q_variables, q_guess)

        result = Solve(prog)

        if running_as_notebook:
            render_context = diagram.CreateDefaultContext()
            plant.SetPositions(
                plant.GetMyContextFromRoot(render_context),
                result.GetSolution(q_variables),
            )
            diagram.ForcedPublish(context)

        if result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None

First, we show an example where we have fixed the base. It may take many tries for it to solve, or not solve at all! At each iteration, we visualize where the optimizer stopped, so you can see what the failures look like.

In [34]:
solve_ik(
    goal_pose,
    max_tries=20,
    fix_base=True,
    base_pose=np.array([-1.23, 0.05, 0]),
)

Succeeded in 5 tries!


array([-1.23000000e+00,  5.00000000e-02,  0.00000000e+00,  1.43042992e-01,
       -2.68006558e-01,  2.81374826e-01, -6.76782283e-02,  0.00000000e+00,
        1.58493515e-06,  0.00000000e+00, -6.14571171e-06,  2.77816907e-01,
       -5.23600000e-01,  6.31783736e-01, -1.22731636e+00,  2.59559086e+00,
       -1.49466691e+00, -5.90297448e-01])

When we allow the base to move freely, the inverse kinematics can be solved much more easily!

In [40]:
result = solve_ik(goal_pose, fix_base=False)
for x in result[:4]:
    print(float(x))

Succeeded in 1 tries!
-1.5291956409979783
0.3430922343648811
-0.4290710370017461
0.31


Below is an autograder where you can check the correctness of your implementation.

In [36]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.mobile.test_mobile_base_ik import TestMobileBaseIk

Grader.grade_output([TestMobileBaseIk], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik is 5/5.
- Succeeded in 1 tries!
Succeeded in 1 tries!
Succeeded in 1 tries!
Failed!



## VERIFICATION IN GRADESCOPE

**Instructions:** Complete the set of exercises below in Gradescope to verify your implementation

## Verification 1: Fixed Base Failures

**Question:** If we vary the x and y coordinates of the base pose for the fixed base case, we notice that sometimes IK fails to find an optimal solution. Which of the following base poses leads to a certain IK failure?

A. [-1.23, 0.05, 0] 
B. [-1.5, -0.2, 0] 
C. [-1.8, 0.2, 0] 
D. [-1.4, 0.3, 0] 


## Verification 2: Optimal Position of the Mobile Base

**Question:** When we set `fix_base=False`, what is the optimal position of the base of the PR2? The first three elements from the optimal solution returned by `solve_ik` represent the xyz coordinates of the base of the PR2. Give your answer to four decimal places. 

**Note:** Be sure you are using the proper tolerances for the orientation and position constraints. The constraints must be precise for the verification.